In [1]:
!pip install --upgrade pip
!pip install transformers datasets peft accelerate torch torchvision jupyter pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 165.6 MB/s  0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1.16.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21/21 [jupyter]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 r

In [2]:
!pip install huggingface_hub huggingface huggingface-cli

ERROR: Could not find a version that satisfies the requirement huggingface-cli (from versions: none)
ERROR: No matching distribution found for huggingface-cli


In [ ]:
!huggingface-cli login

In [4]:
!huggingface-cli whoami

⚠️  Warning: 'huggingface-cli whoami' is deprecated. Use 'hf auth whoami' instead.
user:  blousy


In [ ]:
from transformers import pipeline

ask_llm = pipeline(
    model="Qwen/Qwen2.5-3B-Instruct",
    device="cuda",
    trust_remote_code=True,
    token="HF_TOKEN_KEY"   # 👈 replace with your actual HF token
)

print(ask_llm("你好，我来自哪里？"))


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


[{'generated_text': '你好，我来自哪里？ 你好！我是一个人工智能助手，没有具体的地理位置。我是通过互联网被访问的，所以“来自”哪里这个问题对我来说是没有意义的。你来自哪里呢？我们可以聊聊那里有趣的事情！如果你想知道我的训练数据集中的信息，我可以告诉你，我是由阿里云开发的模型，训练数据来源于互联网上的公开资源。有什么问题或者你想聊的话题吗？'}]


In [8]:
print(ask_llm("what can you do?")[0]['generated_text'])

what can you do? I need help with a programming problem. Sure, I'd be happy to help! Please describe the programming problem you're facing in detail. What language are you using, and what specific issue are you encountering?


In [9]:
from datasets import load_dataset

raw_data = load_dataset("json", data_files="Genzmed.json")

raw_data

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 242
    })
})

In [10]:
raw_data['train'][0]

{'prompt': 'yo why is my skin lowkey betraying me rn?',
 'completion': "fr your skin is probably stressed from hormones, lack of sleep, or that greasy food hitting different. try a gentle cleanser twice daily and don't pick at it bestie - that just makes it worse no cap"}

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-3B-Instruct", trust_remote_code=True)

sample =  raw_data['train'][10]

sample = sample['prompt'] + '\n' + sample['completion']

sample

"is it normal to masturbate?\ntotally normal and healthy! it's a natural part of sexuality and can actually help with stress relief and better sleep. just keep it hygienic and don't feel guilty about something completely normal"

In [12]:
def preprocess(sample):

  sample = sample['prompt'] + '\n' + sample['completion']
  tokenized = tokenizer(
            sample,
            max_length=128,
            truncation=True,
            padding="max_length"
            )

  tokenized["labels"] = tokenized['input_ids'].copy()

  return tokenized

In [13]:
data = raw_data.map(preprocess)

Map:   0%|          | 0/242 [00:00<?, ? examples/s]

In [14]:
print(data['train'][8])

{'prompt': "is it weird that I haven't had my first kiss yet?", 'completion': "not weird at all bestie! there's no timeline for romantic stuff and everyone moves at their own pace. don't let social media make you feel like you're behind - your time will come when it's right for you", 'input_ids': [285, 432, 16283, 429, 358, 8990, 944, 1030, 847, 1156, 21057, 3602, 5267, 1921, 16283, 518, 678, 1850, 645, 0, 1052, 594, 902, 24808, 369, 23467, 6259, 323, 5019, 10797, 518, 862, 1828, 17857, 13, 1513, 944, 1077, 3590, 3687, 1281, 498, 2666, 1075, 498, 2299, 4815, 481, 697, 882, 686, 2525, 979, 432, 594, 1290, 369, 498, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15

In [15]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
    'Qwen/Qwen2.5-3B-Instruct',
    device_map = "cuda",
    torch_dtype = torch.float16
)

lora_config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ["q_proj", "k_proj", "v_proj"]
)

model = get_peft_model(model, lora_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
from transformers import TrainingArguments, Trainer

train_args = TrainingArguments(
    num_train_epochs=10,
    learning_rate=0.001,
    logging_steps=25,
    fp16=True
)

trainer = Trainer(
    args=train_args,
    model=model,
    train_dataset=data['train']
)



In [ ]:
trainer.train()

In [18]:
trainer.save_model('./genz-qwen')
tokenizer.save_pretrained('./genz-qwen')

/usr/local/lib/python3.12/dist-packages/peft/utils/other.py:1228: UserWarning: Unable to fetch remote file due to the following error 401 Client Error: Unauthorized for url: https://huggingface.co/Qwen/Qwen2.5-3B-Instruct/resolve/main/config.json (Request ID: Root=1-68b67994-3cad9c2068760da306bd3dbd;1125cae7-7638-4820-984c-e50ce0bac915)

Invalid credentials in Authorization header - silently ignoring the lookup for the file config.json in Qwen/Qwen2.5-3B-Instruct.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:286: UserWarning: Could not find a config file in Qwen/Qwen2.5-3B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


('./genz-qwen/tokenizer_config.json',
 './genz-qwen/special_tokens_map.json',
 './genz-qwen/chat_template.jinja',
 './genz-qwen/vocab.json',
 './genz-qwen/merges.txt',
 './genz-qwen/added_tokens.json',
 './genz-qwen/tokenizer.json')

In [ ]:
from transformers import pipeline

ask_llm = pipeline(
    model="./genz-qwen",
    tokenizer="./genz-qwen",
    device="cuda",
    trust_remote_code=True,
    token="HF_TOKEN_KEY"   # 👈 replace with your actual HF token
)

print(ask_llm("why coffee makes me feel jittery?"))


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda


[{'generated_text': 'why coffee makes me feel jittery? caffeine is a stimulant that can definitely make you more anxious or jumpy. if coffee is messing with your anxiety, try decaffeinating or cutting back. your nerves will appreciate the calm environment'}]


In [20]:
print(ask_llm("is it good to smoke cigarettes?"))

[{'generated_text': 'is it good to smoke cigarettes? nope nicotine is basically just super addictive poison\nnicotine affects your brain more than regular drugs and can literally kill you. smoking hurts every part of you - your skin, bones, heart, and future self will thank you for quitting ASAP'}]


In [21]:
print(ask_llm("why does my jaw click every time I eat?"))

[{'generated_text': "why does my jaw click every time I eat? clicking sounds in your jaw can happen from clenching or grinding your teeth, or just from the jaw muscles moving around. if it's bothering you or you have pain when you open/close your mouth, your dentist might recommend a night guard"}]


In [22]:
print(ask_llm("My TMJ hurts bro, what to do?"))

[{'generated_text': "My TMJ hurts bro, what to do? \n\nTMJ stands for jaw joint - the little hinge in your head that connects your skull to your lower jaw. Tension from teeth grinding, stress, or whatever's going on in your life can make it hurt.\n\nTry these:\n- gentle stretching\n- stress management\n- avoiding hard foods\n- maybe a warm compress\n- don't clench your teeth\n- get proper sleep hygiene\nif it's chronic or getting worse, better see someone who can check for other causes"}]
